In [39]:
from IPython.display import display
from numpy.random import RandomState
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from IPython.display import display
from numpy.random import RandomState
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC



In [3]:
rs = RandomState(123)

In [4]:
X = pd.read_csv("Dataset.txt", header=None, delimiter=r"\s+",)

In [5]:
X.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

In [6]:
X.isnull().values.any()

True

In [7]:
print(X.head())

   Age  WorkClass  fnlwgt     Education  EducationNum       MaritalStatus  \
0   25    Private  226802          11th             7       Never-married   
1   38    Private   89814       HS-grad             9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm            12  Married-civ-spouse   
3   44    Private  160323  Some-college            10  Married-civ-spouse   
4   18          ?  103497  Some-college            10       Never-married   

          Occupation Relationship   Race  Gender  CapitalGain  CapitalLoss  \
0  Machine-op-inspct    Own-child  Black    Male          0.0          0.0   
1    Farming-fishing      Husband  White    Male          0.0          0.0   
2    Protective-serv      Husband  White    Male          0.0          0.0   
3  Machine-op-inspct      Husband  Black    Male       7688.0          0.0   
4                  ?    Own-child  White  Female          0.0          0.0   

   HoursPerWeek  NativeCountry Income  
0          40.0  United-Stat

In [8]:
y = X["Income"]
X.drop("Income", axis=1, inplace=True,)

In [9]:
X.Age = X.Age.astype(float)
X.fnlwgt = X.fnlwgt.astype(float)
X.EducationNum = X.EducationNum.astype(float)
X.HoursPerWeek = X.HoursPerWeek.astype(float)

In [10]:
y.shape

(8530,)

In [11]:
X.shape

(8530, 14)

In [12]:
X = X.replace('?', np.NaN)

In [13]:
NAs = pd.concat([X.isnull().sum()], axis=1, keys=['Train'])
NAs[NAs.sum(axis=1) > 0]

,Train
WorkClass,515
MaritalStatus,1
Occupation,516
Relationship,1
Race,1
Gender,1
CapitalGain,1
CapitalLoss,1
HoursPerWeek,1
NativeCountry,138


In [14]:
num_attributes = X.select_dtypes(include='float64')

In [15]:
for i in num_attributes:
    X[i] = X[i].fillna(X[i].mean())

In [16]:
cat_attributes = X.select_dtypes(include=['object'])

In [17]:
X = X.drop('EducationNum', axis=1)

In [18]:
for i in cat_attributes:
    X[i] = X[i].fillna(X[i].mode()[0])

In [19]:
y = y.fillna(y.mode()[0])

In [20]:
for col in X.dtypes[X.dtypes == 'object'].index:
    for_dummy = X.pop(col)
    X = pd.concat([X, pd.get_dummies(for_dummy, prefix=col)], axis=1)

In [21]:
y

0       <=50K
1       <=50K
2        >50K
3        >50K
4       <=50K
5       <=50K
6       <=50K
7        >50K
8       <=50K
9       <=50K
10       >50K
11      <=50K
12      <=50K
13      <=50K
14       >50K
15       >50K
16      <=50K
17      <=50K
18      <=50K
19       >50K
20       >50K
21      <=50K
22      <=50K
23      <=50K
24      <=50K
25       >50K
26      <=50K
27      <=50K
28      <=50K
29      <=50K
        ...  
8500    <=50K
8501     >50K
8502    <=50K
8503     >50K
8504     >50K
8505    <=50K
8506    <=50K
8507     >50K
8508    <=50K
8509    <=50K
8510     >50K
8511    <=50K
8512     >50K
8513    <=50K
8514     >50K
8515     >50K
8516     >50K
8517    <=50K
8518    <=50K
8519     >50K
8520     >50K
8521    <=50K
8522    <=50K
8523     >50K
8524    <=50K
8525    <=50K
8526     >50K
8527    <=50K
8528    <=50K
8529    <=50K
Name: Income, Length: 8530, dtype: object

In [22]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state= 123)

# Naive Bayes

In [23]:
gnb = GaussianNB()
model_nb = gnb.fit(x_train,y_train)
model_nb

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
pred_nb = gnb.predict(x_test)
acc = accuracy_score(y_test,pred_nb)
print("The Accuracy of Naive Bayes is ", acc)

The Accuracy of Naive Bayes is  0.8083235638921453


In [26]:
confusion_matrix(y_test,pred_nb)

array([[1253,   71],
       [ 256,  126]])

In [29]:
print(classification_report(y_test, pred_nb))

              precision    recall  f1-score   support

       <=50K       0.83      0.95      0.88      1324
        >50K       0.64      0.33      0.44       382

    accuracy                           0.81      1706
   macro avg       0.73      0.64      0.66      1706
weighted avg       0.79      0.81      0.78      1706



In [52]:
kfold = StratifiedKFold(n_splits = 3, random_state=40)

Cross Val Score

In [53]:
cross_val_score(gnb,X,y,scoring= "accuracy", cv=kfold).mean()

0.7982414509040437

# SVM

In [46]:
svm_ = svm.SVC(kernel='rbf',decision_function_shape='ovr',max_iter=9000)
model_svm = svm_.fit(x_train, y_train)
model_svm

/Users/meryemezber/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=9000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=9000, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [50]:
svm_pred = model.predict(x_test)
acc_svm = accuracy_score(y_test,svm_pred)
print("The Accuracy of SVM is ", acc_svm)

The Accuracy of SVM is  0.7672919109026963


In [54]:
confusion_matrix(y_test, svm_pred)

array([[1301,   23],
       [ 374,    8]])

In [55]:
print(classification_report(y_test, svm_pred))

              precision    recall  f1-score   support

       <=50K       0.78      0.98      0.87      1324
        >50K       0.26      0.02      0.04       382

    accuracy                           0.77      1706
   macro avg       0.52      0.50      0.45      1706
weighted avg       0.66      0.77      0.68      1706



Cross Val Score

In [56]:
cross_val_score(svm_,X,y,scoring= "accuracy", cv=kfold).mean()

/Users/meryemezber/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=9000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/meryemezber/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=9000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/meryemezber/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=9000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


0.7622515129530157

# Second Part of HW

In [51]:
tree = DecisionTreeClassifier()
model_tree = tree.fit(x_train,y_train)
model_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [53]:
model_tree = tree.fit(x_train,y_train)
pred_tree = tree.predict(x_test)
a1 = accuracy_score(y_test,pred_tree)
print("The Accuracy of Desicion Tree is ", a1)

The Accuracy of Desicion Tree is  0.8143806174286831


In [54]:
confusion_matrix(y_test,pred_tree)

array([[1704,  264],
       [ 211,  380]])

In [55]:
print(classification_report(y_test, pred_tree))

              precision    recall  f1-score   support

       <=50K       0.89      0.87      0.88      1968
        >50K       0.59      0.64      0.62       591

    accuracy                           0.81      2559
   macro avg       0.74      0.75      0.75      2559
weighted avg       0.82      0.81      0.82      2559



# Random Forest Model with Default parameters¶

In [58]:
rf = RandomForestClassifier()
model_rf = rf.fit(x_train,y_train)
pred_rf = rf.predict(x_test)
a2 = accuracy_score(y_test, pred_rf)
print("The Accuracy of Random Forest is ", a2)

The Accuracy of Random Forest is  0.8440797186400938


# Logistic Regression 

In [59]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()

model_lg = lg.fit(x_train,y_train)
pred_lg = lg.predict(x_test)
a3 = accuracy_score(y_test, pred_lg)
print("The Accuracy of logistic regression is ", a3)

The Accuracy of logistic regression is  0.8120359515435717


# KNN model

In [60]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier()

In [61]:
model_knn =knn.fit(x_train,y_train) 
pred_knn = knn.predict(x_test)
a4 = accuracy_score(y_test, pred_knn)
print("The Accuracy of KNN is ", a4)

The Accuracy of KNN is  0.776084407971864


# Grid search for RF

In [62]:
rf_param = {
    "n_estimators": [25,50,100],
    "criterion" : ["gini"],
    "max_depth" : [3,4,5,6],
    "max_features" : ["auto","sqrt","log2"],
    "random_state" : [123]
}

In [63]:
GridSearchCV(rf, rf_param, cv = 5)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [64]:
grid =GridSearchCV(rf, rf_param, cv = 5)

In [65]:
grid.fit(x_train,y_train).best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'auto',
 'n_estimators': 50,
 'random_state': 123}

In [66]:
rf1 = RandomForestClassifier(criterion = 'gini',
    max_depth = 6,
    max_features = 'auto',
    n_estimators = 100,
    random_state = 123)
model_rf1 = rf1.fit(x_train,y_train)
pred_rf1 = rf1.predict(x_test)
accuracy_score(y_test, pred_rf1)

0.8460336068776866

# Grid search for LG

In [117]:
lg_param = {'penalty': ['l1', 'l2'],
            'C':[0.001,.009,0.01,.09,1,5,10,25]}

In [118]:
lg_grid =GridSearchCV(lg, lg_param, cv = kfold)

In [119]:
lg_grid.fit(x_train,y_train).best_params_

{'C': 1, 'penalty': 'l1'}

In [120]:
lg2 = LogisticRegression(C = 1, penalty= 'l1')
model_lg2 = lg2.fit(x_train,y_train)
pred_lg2 = lg2.predict(x_test)
accuracy_score(y_test, pred_lg2)

0.8515044939429465

# K FOLD Cross Validation

In [155]:
kfold = StratifiedKFold(n_splits = 3, random_state=40)

In [156]:
cross_val_score(tree,X,y,scoring= "accuracy", cv=kfold)

array([0.80625879, 0.81012658, 0.80577058])

In [157]:
cross_val_score(tree,X,y,scoring= "accuracy", cv=kfold).mean()

0.8082056771493699

In [158]:
cross_val_score(rf,X,y,scoring= "accuracy", cv=kfold).mean()

0.8406797706230269

In [159]:
cross_val_score(lg,X,y,scoring= "accuracy", cv=kfold).mean()

0.8034000738371879

In [160]:
cross_val_score(knn,X,y,scoring= "accuracy", cv=kfold).mean()

0.7747952568679638

# Voting Classifier model

In [73]:
from sklearn.ensemble import VotingClassifier

In [161]:
model_vote = VotingClassifier(estimators=[('logistic Regression', lg), ('random forrest', rf), ('knn neighbors', knn),(' decision tree', tree)], voting='soft')
model_vote = model_vote.fit(x_train, y_train)

In [162]:
vote_pred = model_vote.predict(x_test)

In [163]:
a5 =  accuracy_score(y_test, vote_pred)
print("The Accuracy of voting classifier is ", a5)

The Accuracy of voting classifier is  0.8436889409925752


In [164]:
cross_val_score(model_vote,x_train,y_train,scoring= "accuracy", cv=5).mean()

0.837216678883445

In [134]:
kfold = StratifiedKFold(n_splits = 3, random_state=40)

# Bagging Classifier

In [135]:
from sklearn.ensemble import BaggingClassifier

In [136]:
bagg = BaggingClassifier(base_estimator=rf1,n_estimators=15)

In [137]:
model_bagg =bagg.fit(x_train,y_train) 
pred_bagg = bagg.predict(x_test)

In [138]:
a6 = accuracy_score(y_test, pred_bagg)
print("The Accuracy of BAAGING is ", a6)

The Accuracy of BAAGING is  0.8417350527549824


In [139]:
cross_val_score(model_bagg,x_train,y_train,scoring= "accuracy", cv=kfold).mean()

0.8291746410043452

In [140]:
lg_bagg = BaggingClassifier(base_estimator=lg2,n_estimators=15)

In [141]:
lg_model_bagg =lg_bagg.fit(x_train,y_train) 
lg_pred_bagg = lg_bagg.predict(x_test)

In [142]:
a10 = accuracy_score(y_test, lg_pred_bagg)
print("The Accuracy of BAAGING is ", a10)

The Accuracy of BAAGING is  0.8507229386479094


In [143]:
cross_val_score(lg_model_bagg,x_train,y_train,scoring= "accuracy", cv=kfold).mean()

0.8455882037292503

# ADABOOST Classifier

In [144]:
from sklearn.ensemble import AdaBoostClassifier

In [145]:
Adaboost = AdaBoostClassifier(base_estimator=rf1, n_estimators=15)

In [146]:
model_boost =Adaboost.fit(x_train,y_train) 
pred_boost = Adaboost.predict(x_test)

In [147]:
a7 = accuracy_score(y_test, pred_boost)
print("The Accuracy of BOOSTING is ", a7)

The Accuracy of BOOSTING is  0.8620554904259476


In [148]:
cross_val_score(lg_model_boost,x_train,y_train,scoring= "accuracy", cv=kfold).mean()

0.8554698590447697

In [149]:
lg_Adaboost = AdaBoostClassifier(base_estimator=lg2, n_estimators=15)

In [150]:
lg_model_boost =Adaboost.fit(x_train,y_train) 
lg_pred_boost = Adaboost.predict(x_test)

In [151]:
a11 = accuracy_score(y_test, lg_pred_boost)
print("The Accuracy of BAAGING is ", a11)

The Accuracy of BAAGING is  0.8604923798358733


In [152]:
cross_val_score(lg_model_boost,x_train,y_train,scoring= "accuracy", cv=kfold).mean()

0.8554695223528502

In [153]:
confusion_matrix(y_test,pred_boost)

array([[1846,  122],
       [ 231,  360]])

In [154]:
print(classification_report(y_test, pred_boost))

              precision    recall  f1-score   support

       <=50K       0.89      0.94      0.91      1968
        >50K       0.75      0.61      0.67       591

    accuracy                           0.86      2559
   macro avg       0.82      0.77      0.79      2559
weighted avg       0.86      0.86      0.86      2559

